<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/butterfly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vizro -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.0/857.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.8/500.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 8.1 MB/s eta 0:00:00


In [ ]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata


engine = create_engine(userdata.get("supabase"), client_encoding='utf8', )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [ ]:
sql = """
SELECT *, cnt * 100 /platform_cnt as pct FROM (
SELECT platform, query, count(*) as cnt,
row_number() OVER (PARTITION BY platform ORDER BY COUNT(*) DESC) as rnb,
sum(count(*)) OVER (PARTITION BY platform) as  platform_cnt
FROM vizro.yandex_data
GROUP BY platform, query) t
WHERE rnb < 11
ORDER BY rnb

"""
query_df = select(sql)

In [ ]:
sql = """
SELECT platform, query, count(*) FROM vizro.yandex_data WHERE query in (
SELECT DISTINCT query FROM (
SELECT platform, query, count(*) as cnt,
row_number() OVER (PARTITION BY platform ORDER BY COUNT(*) DESC) as rnb

FROM vizro.yandex_data
GROUP BY platform, query) t
WHERE rnb < 11)
GROUP BY platform, query


"""
df = select(sql)

In [ ]:
df

,platform,query,count
0,desktop,xxx,261
1,touch,календарь 2021,2268
2,desktop,английский алфавит,1352
3,desktop,доброе утро,145
4,desktop,алфавит,923
5,touch,хентай,4194
6,desktop,таблица квадратов,883
7,touch,доброе утро,2342
8,desktop,с днём рождения,297
9,touch,обои на рабочий стол,102


In [ ]:
print_data = df.pivot(index='query', columns='platform', values='count').reset_index().sort_values(by='touch', ascending=True)

In [ ]:
print_data

platform,query,desktop,touch
8,одноклассники (социальная сеть),1116,5
7,обои на рабочий стол,1142,102
15,таблица квадратов,883,587
2,алфавит,923,1225
3,английский алфавит,1352,1748
6,картинки,1720,1801
5,календарь 2021,2809,2268
16,таблица менделеева,2659,2278
4,доброе утро,145,2342
0,xnxx,62,2767


In [ ]:
sql = """
SELECT date, date_part('hour', ts)::int as hour, platform, query
FROM vizro.yandex_data
"""
query_df = select(sql)

In [ ]:
query_df['date'] = pd.to_datetime(query_df['date'],  format='%y-%m-%d')

In [ ]:
total = query_df.groupby("platform")["date"].count().to_dict()

In [ ]:
filtered = query_df[query_df['query'].isin(set(query_df.groupby(['platform', 'query'], as_index=False)['date'].count().sort_values(by='date',ascending=False).groupby('platform', as_index=False).head(10)["query"]))]


In [ ]:
filtered_group = filtered.groupby(['platform', 'query'])["date"].count().unstack(level=0)
filtered_group.reset_index(inplace=True)

In [ ]:
for k, v in total.items():
    filtered_group[f'{k}_%'] = filtered_group[k] / v


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import vizro.models as vm
from vizro import Vizro
from vizro.models.types import capture


@capture("graph")
def butterfly(data_frame: pd.DataFrame, **kwargs) -> go.Figure:
    fig = px.bar(data_frame, **kwargs)

    orientation = fig.data[0].orientation
    x_or_y = "x" if orientation == "h" else "y"

    fig.update_traces({f"{x_or_y}axis": f"{x_or_y}2"}, selector=1)
    fig.update_layout({f"{x_or_y}axis2": fig.layout[f"{x_or_y}axis"]})
    fig.update_layout(
        {
            f"{x_or_y}axis": {"autorange": "reversed", "domain": [0, 0.5]},
            f"{x_or_y}axis2": {"domain": [0.5, 1]},
        }
    )

    if orientation == "h":
        fig.add_vline(x=0, line_width=2, line_color="grey")
    else:
        fig.add_hline(y=0, line_width=2, line_color="grey")
    fig.data[0].hovertemplate = '<b>%{hovertext}</b><br>% of all=%{x:.2%}<br>qty=%{customdata[0]}<extra></extra>'
    fig.data[0].name = 'desktop'
    fig.data[1].hovertemplate = '<b>%{hovertext}</b><br>% of all=%{x:.2%}<br>qty=%{customdata[1]}<extra></extra>'
    fig.data[1].name = 'touch'
    fig.update_yaxes(categoryorder='min ascending')

    return fig



fig = butterfly(
    filtered_group,
    x=["desktop_%", "touch_%"],
    y="query",
    labels={"value": "% of all", "variable": "platform:"},
    hover_name="query", hover_data={'query': False, 'desktop': True, 'touch': True },
)

page = vm.Page(title="My page", components=[vm.Graph(figure=fig)])
dashboard = vm.Dashboard(pages=[page])
Vizro().build(dashboard).run()

<IPython.core.display.Javascript object>

In [ ]:
fig.data[0].name = 'desktop'

In [ ]:
fig